In [ ]:
pip install konlpy

     |████████████████████████████████| 19.4MB 6.4MB/s 
     |████████████████████████████████| 2.7MB 37.5MB/s 


In [ ]:
string = []
tag = []
with open("./ratings_train.txt",'r',encoding='utf-8-sig') as f:
    cnt = 0
    for line in f.readlines():
        if cnt == 0:
            pass
        else:
            line = line.split('\t')
            string.append(line[1])
            tag.append(int(line[2][0]))
        cnt += 1

from konlpy.tag import Okt
twitter = Okt()

string_morphs = []
string_char = []

max_morph_len = 0

morph_set = set()
for line in string:
    morph = twitter.pos(line)
    morphs = []
    for mor in morph:
        if mor[1] != "Josa":
            morphs.append(mor[0])
    string_morphs.append(morphs)
    max_morph_len = max_morph_len if max_morph_len > len(morphs) else len(morphs)
    morph_set.update(morphs)

/usr/local/lib/python3.6/dist-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [ ]:
input_morph = []

morph_set = list(['<p>']) + list(morph_set)
morph_set = {morph:index for index,morph in enumerate(morph_set)}
for line in string_morphs:
    morph = [morph_set[mor] for mor in line]
    input_morph.append(morph)
from keras.preprocessing.sequence import pad_sequences
input_morph = pad_sequences(input_morph ,padding = 'post'
    , maxlen = max_morph_len)

from keras.utils import to_categorical
tag = to_categorical(tag)

from sklearn.model_selection import train_test_split
input_morph_train,input_morph_val,tag_train,tag_val = train_test_split(input_morph,tag,test_size = 0.1)

Using TensorFlow backend.


In [ ]:
from keras.layers import Input, Dense, LSTM,Conv1D,Embedding,Reshape,GlobalMaxPooling1D,Flatten
from keras.models import Model

input_morph = Input((max_morph_len,))
embed_output = Embedding(len(morph_set),
                         100)(input_morph)
cnn_output = Conv1D(32,kernel_size = 3,activation = 'relu')(lstm_output)
cnn_pool = GlobalMaxPooling1D()(cnn_output)
output = Dense(2,activation = 'softmax')(cnn_pool)
model = Model([input_morph],output)
model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
import keras

callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss',
                                           patience=1)]
model.fit(input_morph_train,tag_train,
         epochs = 10,
         batch_size = 256,
         validation_data = (input_morph_val,tag_val),
         callbacks = callbacks
         )


Train on 135000 samples, validate on 15000 samples
Epoch 1/10
135000/135000 [==============================] - 113s 835us/step - loss: 0.3841 - acc: 0.8248 - val_loss: 0.3282 - val_acc: 0.8563
Epoch 2/10
135000/135000 [==============================] - 110s 812us/step - loss: 0.2318 - acc: 0.9072 - val_loss: 0.3445 - val_acc: 0.8558


In [ ]:
test_labels = []
test_strings = []
with open('./ratings_test.txt','r',encoding='utf-8-sig') as f:
    cnt = 0
    for line in f.readlines():
        if cnt ==0:
            pass
        else:
            line = line.split('\t')
            test_label = int(line[2][0])
            test_strings.append(line[1])
            test_labels.append(test_label)
        cnt +=1
test_morph = []
for line in test_strings:
    morph = twitter.pos(line)
    morphs = []
    for mor in morph:
        if mor[1] != 'Josa':
            morphs.append(mor[0])
    step_morph = [morph_set[m] if m in morph_set else 0 for m in morphs]
    test_morph.append(step_morph)

test_morph = pad_sequences(test_morph,padding = 'post',maxlen = max_morph_len)

In [ ]:
point = model.evaluate([test_morph],test_labels)
print(point)

50000/50000 [==============================] - 112s 2ms/step
[0.3492832522153854, 0.8527]
